In [0]:
from google.colab import drive


In [0]:
drive.mount('/content/gdrive')


In [0]:
import pandas as pd
census= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Data_sets/census_data.csv')

In [5]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [0]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [0]:
from sklearn.model_selection import train_test_split
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

In [0]:
import tensorflow as tf

In [11]:
census.dtypes

age                int64
workclass         object
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
gender            object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income_bracket     int64
dtype: object

In [0]:
census.head()

In [0]:
gender=tf.feature_column.categorical_column_with_vocabulary_list('gender',['Male','Female'])
workclass=tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket('education',hash_bucket_size=1000)
marital_status=tf.feature_column.categorical_column_with_hash_bucket('marital_status',hash_bucket_size=1000)
occupation=tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
race=tf.feature_column.categorical_column_with_hash_bucket('race',hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket('relationship',hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket('native_country',hash_bucket_size=1000)

In [0]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [0]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [0]:
input_fun=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [22]:
model_lnn = tf.estimator.LinearClassifier(feature_columns=feat_cols)
model_dnn = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpacb0etcx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde6be05ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm9k0idet', '_tf_random_seed': None, '_save_summar

In [0]:
model_lnn.train(input_fn=input_fun,steps=5000)
model_dnn.train(input_fn=input_fun,steps=5000)

In [25]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
predictions = list(model_lnn.predict(input_fn=pred_fn))
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpacb0etcx/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [26]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,final_preds))
print('\n')
print(accuracy_score(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      7436
           1       0.64      0.69      0.67      2333

   micro avg       0.83      0.83      0.83      9769
   macro avg       0.77      0.79      0.78      9769
weighted avg       0.84      0.83      0.84      9769



0.8347834988228069


In [0]:
gender = tf.feature_column.embedding_column(gender, dimension=2)
workclass = tf.feature_column.embedding_column(workclass, dimension=1000)
education = tf.feature_column.embedding_column(education, dimension=1000)
marital_status = tf.feature_column.embedding_column(marital_status, dimension=1000)
occupation = tf.feature_column.embedding_column(occupation, dimension=1000)
relationship = tf.feature_column.embedding_column(relationship, dimension=1000)
race = tf.feature_column.embedding_column(race, dimension=1000)
native_country = tf.feature_column.embedding_column(native_country, dimension=1000)

feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

input_fun=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

model_dnn = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols)

model_dnn.train(input_fn=input_fun,steps=5000)




In [31]:
X_test.shape

(9769, 13)

In [0]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)


In [37]:
predictions_dnn = list(model_dnn.predict(input_fn=pred_fn))
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvadxxtmd/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
len(final_preds)

9769

In [39]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,final_preds))
print('\n')
print(accuracy_score(y_test,final_preds))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      7436
           1       0.64      0.69      0.67      2333

   micro avg       0.83      0.83      0.83      9769
   macro avg       0.77      0.79      0.78      9769
weighted avg       0.84      0.83      0.84      9769



0.8347834988228069


In [0]:
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()
#model_dnn,'/content/gdrive/My Drive/Colab Notebooks/save_sess/tf_class_dnn'

In [0]:
save_path = saver.save(model_dnn,'/content/gdrive/My Drive/Colab Notebooks/save_sess/tf_class_dnn.ckpt')
